# KAKENから研究課題のXMLファイルをAPI経由でダウンロードするプログラム

## 事前準備

+ CiNiiウェブAPIの利用登録を済ませておく。https://support.nii.ac.jp/ja/cinii/api/developer
+ 登録が完了したら、アプリケーションID(appid)がメールで通知される
+ configparserで別途appidを設定ファイルに書き込んでおく。下のセルは、サンプルのコードです。自分は、セル1個だけの別のnotebookを作っています。

In [ ]:
# encoding: utf-8
import configparser

config = configparser.ConfigParser()
config['mariadb'] = {}
config['CiNii_web_api'] = {}

# ローカルのMariaDB
config['mariadb']['username'] = ''
config['mariadb']['password'] = ''
config['mariadb']['database'] = ''

# CiNii ウェブ API
config['CiNii_web_api']['appid'] = '' # 右辺にappid文字列を指定

with open('config.ini', 'w') as configfile:
    config.write(configfile)

## ここから本編

各種ライブラリをインポートする

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm_notebook
import configparser

config.iniから設定項目を読み出す

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
appid = config['CiNii_web_api']['appid']

- 前半：スクレイピングで年度ごとの件数を取得する。
- 後半：KAKEN API用URLを作成してレスポンスを返し、XMLファイルを保存する。
- 1行目のrangeで適宜の年度を指定してください。
- カレントディレクトリにxmlフォルダが作成され、そこにXMLファイルが保存されます。

ほんとは、関数を定義するのが普通ですが、以前にこれで作って動くので、ひとまずこのまま公開します。時間があれば手直ししたいところ。

In [ ]:
for year in tqdm_notebook(range(1964, 2018)):
    KAKEN_ROOT_URL = 'https://kaken.nii.ac.jp/opensearch/?appid=' + appid
    rw = 500

    """ある年度の件数を取得する。"""
    #URL生成
    url = KAKEN_ROOT_URL + '&format=xhtml' + '&s1=' + str(year) + '&s2=' + str(year) + '&o1=1'

    r = requests.get(url)
    html = r.text

    soup = BeautifulSoup(html, "lxml")
    tag = soup.select("p.search-term-selected")
    p = str(tag[0])

    #数字＝件数を抽出する
    pattern = r'([+-]?[0-9]+\,?[0-9]*)'
    numberlist = re.findall(pattern,p)
    cnt = str(numberlist[0])
    cnt = int(cnt.replace(',', ''))
    #cnt
    
    for start in tqdm_notebook(range(1, cnt, rw)):
        """KAKEN API用URLを作成してレスポンスを返す。ファイルを保存する"""
        if start + rw > cnt:
            end = cnt
        else:
            end = start + rw - 1

        url = KAKEN_ROOT_URL + '&format=xml' + '&s1=' + str(year) + '&s2=' + str(year) + '&o1=1' + '&st=' + str(start) + '&rw=' + str(rw)
        r = requests.get(url)
        if r.status_code == 200:
            filename = 'xml/' + str(year) + '_' + str(start) + '-' + str(end) + '.xml'
            with open(filename, mode='w') as f:
                f.write(r.text)

## おしまい

xmlフォルダを確認し、各年度のファイルが保存されていればOK！